# Imports & Seeds 

In [3]:
# === 0. Imports & Seeds ===
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Input, Conv1D, MaxPooling1D, LSTM, Dense, Dropout,
                                     BatchNormalization, Layer)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Custom Layers & Loss

In [4]:
# === 1. Custom Layers & Loss ===
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()
    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)
    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + K.epsilon()))
    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

In [5]:
class FocalLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=0.25, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    def call(self, y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon())
        ce = -y_true * K.log(y_pred)
        loss = self.alpha * K.pow(1.0 - y_pred, self.gamma) * ce
        return K.mean(loss, axis=-1)

In [18]:
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='att_weight',
                                 shape=(input_shape[-1], input_shape[-1]),
                                 initializer='glorot_uniform',
                                 trainable=True)
        self.b = self.add_weight(name='att_bias',
                                 shape=(input_shape[-1],),
                                 initializer='zeros',
                                 trainable=True)
        self.u = self.add_weight(name='att_u',
                                 shape=(input_shape[-1], 1),  # ✅ 2D shape!
                                 initializer='glorot_uniform',
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))  # shape: (batch, timesteps, features)
        ait = K.dot(uit, self.u)                            # shape: (batch, timesteps, 1)
        ait = K.squeeze(ait, -1)                            # shape: (batch, timesteps)
        a = K.softmax(ait)                                  # shape: (batch, timesteps)
        a = K.expand_dims(a, -1)                            # shape: (batch, timesteps, 1)
        return K.sum(x * a, axis=1)                         # shape: (batch, features)


# Load & Preprocess Data

In [8]:
can_data = pd.read_csv(r"G:\road\signal_extractions\attacks\correlated_signal_attack_1_masquerade.csv")
can_data.fillna(0, inplace=True)

signal_columns = [col for col in can_data.columns if "Signal" in col]
can_data['Time'] = pd.to_datetime(can_data['Time'], unit='s')
can_data = can_data.sort_values('Time').set_index('Time')
can_data[signal_columns] = can_data[signal_columns].apply(pd.to_numeric, errors='coerce')
can_data = can_data.resample('100us').mean().interpolate(method='cubic').reset_index()


#  Sliding Window Creation

In [9]:
def create_time_series(data, window_size=10, step_size=1):
    sequences, labels = [], []
    for i in range(0, len(data) - window_size, step_size):
        window = data.iloc[i:i + window_size]
        sequences.append(window[signal_columns].values)
        label = 1 if window['Label'].sum() > 0 else 0  # Majority or any attack
        labels.append(label)
    return np.array(sequences), np.array(labels)

X, y = create_time_series(can_data)
X = np.nan_to_num(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


# Scaling & Balancing

In [10]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)


Oversampling minority

In [13]:
train_df = pd.DataFrame(X_train.reshape(X_train.shape[0], -1))
train_df['label'] = y_train
maj = train_df[train_df['label'] == 0]
min_ = train_df[train_df['label'] == 1]
min_upsampled = resample(min_, replace=True, n_samples=len(maj), random_state=42)
balanced = pd.concat([maj, min_upsampled])
X_train = balanced.drop('label', axis=1).values.reshape(-1, X_train.shape[1], X_train.shape[2])
y_train = balanced['label'].values

input_shape = (X_train.shape[1], X_train.shape[2])

# Model Architectures

In [14]:
def create_cnn_lstm_model(input_shape):
    model = Sequential([
        Conv1D(64, 3, activation='relu', padding='same', input_shape=input_shape),
        MaxPooling1D(2),
        Dropout(0.2),
        LSTM(64, return_sequences=True),
        Dropout(0.3),
        LSTM(32),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    return model

In [15]:
def create_attention_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv1D(64, 3, activation='relu', padding='same')(inputs)
    x = MaxPooling1D(2)(x)
    x = LSTM(64, return_sequences=True)(x)
    x = Dropout(0.3)(x)
    x = LSTM(32, return_sequences=True)(x)
    x = AttentionLayer()(x)
    x = Dense(32, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)
    return Model(inputs, output)

# Training

In [16]:
models = {}
EPOCHS = 20
BATCH_SIZE = 64
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [21]:
for name, model_func in {
    'cnn_lstm': create_cnn_lstm_model,
    'attention': create_attention_model
}.items():
    print(f"\n--- Training {name.upper()} ---")
    model = model_func(input_shape)
    model.compile(optimizer=Adam(0.001, clipnorm=1.0), loss='binary_crossentropy',
                  metrics=['accuracy', Precision(), Recall(), F1Score()])
    model.fit(X_train, y_train, validation_data=(X_test, y_test),
              epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stop], verbose=2)
    models[name] = model


--- Training CNN_LSTM ---
Epoch 1/20
5162/5162 - 63s - loss: 0.6921 - accuracy: 0.5097 - precision_6: 0.5112 - recall_6: 0.4429 - f1_score: 0.4746 - val_loss: 0.6940 - val_accuracy: 0.3763 - val_precision_6: 0.3763 - val_recall_6: 1.0000 - val_f1_score: 0.5468 - 63s/epoch - 12ms/step
Epoch 2/20
5162/5162 - 62s - loss: 0.6927 - accuracy: 0.5075 - precision_6: 0.5074 - recall_6: 0.5104 - f1_score: 0.5089 - val_loss: 0.6871 - val_accuracy: 0.6238 - val_precision_6: 1.0000 - val_recall_6: 8.0292e-05 - val_f1_score: 1.6057e-04 - 62s/epoch - 12ms/step
Epoch 3/20
5162/5162 - 61s - loss: 0.6880 - accuracy: 0.5275 - precision_6: 0.5399 - recall_6: 0.3719 - f1_score: 0.4404 - val_loss: 0.6911 - val_accuracy: 0.6253 - val_precision_6: 0.7550 - val_recall_6: 0.0061 - val_f1_score: 0.0120 - 61s/epoch - 12ms/step
Epoch 4/20
5162/5162 - 72s - loss: 0.6806 - accuracy: 0.5469 - precision_6: 0.5756 - recall_6: 0.3567 - f1_score: 0.4405 - val_loss: 0.6929 - val_accuracy: 0.6239 - val_precision_6: 0.6170

In [22]:
# print label distributions
print("\nLabel Distribution in Training Set:")
print(balanced['label'].value_counts(normalize=True))



Label Distribution in Training Set:
label
0    0.5
1    0.5
Name: proportion, dtype: float64
